# RomExchange API Sandbox

API Docs : https://jessicayeh.github.io/rom-exchange-openapi/#section/Recent-Changes

In [3]:
import requests

In [52]:
r = requests.get(url, headers=headers)

https://www.romexchange.com/api/


In [56]:
r.text

'<head><title>Not Acceptable!</title></head><body><h1>Not Acceptable!</h1><p>An appropriate representation of the requested resource could not be found on this server. This error was generated by Mod_Security.</p></body></html>'